In [1]:
import pandas as pd
from pathlib import Path
import sys
%matplotlib inline

In [2]:
# Set the file path for the CSV using the Path class from the pathlib library
opexpath = "Gross operating exp..csv"
payoutpath = "gross payout.csv"
prempath = "gross premiums.csv"

In [3]:
# pull out operating expenses labeled "All undertakings (=1+3)" in "Ownership" column and "TOT" in "DBRA" column
# add year and value to main_df
opexdf = pd.read_csv(opexpath)
opexdf = opexdf[opexdf["Ownership"] == "All undertakings (=1+3)"]
opexdf = opexdf[opexdf["DBRA"] == "TOT"]
main_df = opexdf[["Year","Value"]].reset_index().drop('index',axis=1)
main_df.columns =["Year","Operating Expenses"]

# pull out gross payouts labeled "All undertakings (=1+3)" in "Ownership" column and "TOT" in "DBRA" column
# add column to main_df
payoutdf = pd.read_csv(payoutpath)
payoutdf = payoutdf[payoutdf["Ownership"] == "All undertakings (=1+3)"]
payoutdf = payoutdf[payoutdf["DBRA"] == "TOT"]
main_df["Gross Payout"] = payoutdf["Value"].reset_index().drop('index',axis=1)

# pull out gross premiums labeled "I_2" in "IND" column
# add column to main_df
premdf = pd.read_csv(prempath)
premdf = premdf[premdf["IND"] == "I_2"]
main_df["Gross Premiums"] = premdf["Value"].reset_index().drop('index',axis=1)
main_df = main_df.set_index("Year")
main_df
yearly_chng = main_df.pct_change().dropna()
yearly_chng.mean()

Operating Expenses    0.006834
Gross Payout          0.033024
Gross Premiums        0.036030
dtype: float64

In [4]:
# create lists for new df to project exp, payouts, prems into the future
# rate of increase is based on avg increase since 2008
years = [main_df.index[-1]]
opex = [main_df["Operating Expenses"].iloc[-1]]
payouts = [main_df["Gross Payout"].iloc[-1]]
prems = [main_df["Gross Premiums"].iloc[-1]]
surplus = [prems[0] - payouts[0] - opex[0]]
percentage_surplus = [surplus[0] * 100 / prems[0]]           
counter = 1



In [5]:
while counter <= 30:
    
    current_year = years[-1] + 1
    current_opex = opex[-1] * (1 + yearly_chng["Operating Expenses"].mean()) 
    current_payout = payouts[-1] * (1 + yearly_chng["Gross Payout"].mean()) 
    current_prem = prems[-1] * (1 + yearly_chng["Gross Premiums"].mean()) 
    current_surplus = current_prem - current_payout - current_opex 
    current_percentage_surplus = current_surplus * 100 / current_prem 
    
    years.append(current_year)
    opex.append(current_opex)
    payouts.append(current_payout)
    prems.append(current_prem)
    surplus.append(current_surplus)
    percentage_surplus.append(current_percentage_surplus)
      
    counter += 1

project_df = pd.DataFrame({"Years" : years, 
                           "Operating Expenses" : opex,
                           "Gross Payout" : payouts, 
                           "Gross Premiums" : prems, 
                           "Surplus" : surplus,
                           "% Surplus vs Gross Premiums" : percentage_surplus}).set_index("Years")

project_df

,Operating Expenses,Gross Payout,Gross Premiums,Surplus,% Surplus vs Gross Premiums
Years,,,,,
2018,75903.850000,4.362193e+05,1.139513e+06,6.273897e+05,55.057714
2019,76422.568116,4.506251e+05,1.180569e+06,6.535218e+05,55.356487
2020,76944.831092,4.655066e+05,1.223105e+06,6.806538e+05,55.649649
2021,77470.663152,4.808796e+05,1.267174e+06,7.088233e+05,55.937349
2022,78000.088688,4.967603e+05,1.312830e+06,7.380694e+05,56.219732
2023,78533.132256,5.131654e+05,1.360131e+06,7.684324e+05,56.496941
2024,79069.818583,5.301123e+05,1.409136e+06,7.999542e+05,56.769112
2025,79610.172561,5.476189e+05,1.459907e+06,8.326784e+05,57.036381
2026,80154.219257,5.657036e+05,1.512508e+06,8.666500e+05,57.298877
